5. Design and implement Deep Convolutional GAN to generate images of faces/digits from a set of
given images.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, LeakyReLU, Conv2D, Conv2DTranspose
from tensorflow.keras.optimizers import Adam

2025-04-22 14:16:48.429408: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-22 14:16:48.431880: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-22 14:16:48.436488: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-22 14:16:48.448697: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745311608.482896  172253 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745311608.48

In [2]:
# Load MNIST dataset (digits)
(X_train, _), (_, _) = mnist.load_data()

# Normalize to [-1, 1] for better training of GAN
X_train = (X_train - 127.5) / 127.5  
X_train = X_train.reshape(-1, 28, 28, 1)  # Reshape to (28x28x1) for CNN input

In [3]:
def make_generator():
    model = Sequential([
        Dense(7*7*128, input_dim=100),  # Fully connected layer to reshape
        LeakyReLU(0.2),  # LeakyReLU activation
        Reshape((7, 7, 128)),  # Reshape into 7x7x128
        Conv2DTranspose(64, (4,4), strides=2, padding='same', activation='relu'),  # Upsample
        Conv2DTranspose(1, (4,4), strides=2, padding='same', activation='tanh')  # Final output layer
    ])
    return model

In [4]:
def make_discriminator():
    model = Sequential([
        Conv2D(64, (3,3), strides=2, padding='same', input_shape=(28,28,1)),  # Convolutional layer
        LeakyReLU(0.2),  # LeakyReLU activation
        Flatten(),  # Flatten to 1D
        Dense(1, activation='sigmoid')  # Output real/fake classification
    ])
    model.compile(optimizer=Adam(0.0002), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [5]:
# Create models
generator = make_generator()
discriminator = make_discriminator()

# Freeze discriminator weights while training the GAN
discriminator.trainable = False

# Build GAN model by stacking generator and discriminator
gan = Sequential([generator, discriminator])
gan.compile(optimizer=Adam(0.0002), loss='binary_crossentropy')

/home/pl1/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-04-22 14:17:20.814058: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/pl1/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
def train(epochs=10000, batch=64):
    for epoch in range(epochs):
        # --- Train the Discriminator ---
        idx = np.random.randint(0, X_train.shape[0], batch)
        real = X_train[idx]  # Get real images
        noise = np.random.normal(0, 1, (batch, 100))  # Generate random noise
        fake = generator.predict(noise, verbose=0)  # Generate fake images

        # Discriminator loss on real images (label = 1)
        d_loss_real = discriminator.train_on_batch(real, np.ones((batch, 1)))
        # Discriminator loss on fake images (label = 0)
        d_loss_fake = discriminator.train_on_batch(fake, np.zeros((batch, 1)))
        
        # --- Train the Generator ---
        noise = np.random.normal(0, 1, (batch, 100))  # Generate noise for generator
        g_loss = gan.train_on_batch(noise, np.ones((batch, 1)))  # Train the generator to fool the discriminator

        # Print losses at every 1000 epochs
        if epoch % 1000 == 0:
            print(f"{epoch} | D loss: {np.mean([d_loss_real[0], d_loss_fake[0]]):.4f} | G loss: {g_loss:.4f}")
            show_images(generator)  # Display generated images periodically


In [7]:
def show_images(gen_model, examples=25):
    noise = np.random.normal(0, 1, (examples, 100))  # Generate noise
    gen_imgs = gen_model.predict(noise, verbose=0)  # Generate images from noise
    gen_imgs = 0.5 * gen_imgs + 0.5  # Rescale to [0, 1] for display

    # Plot the generated images
    plt.figure(figsize=(5,5))
    for i in range(examples):
        plt.subplot(5,5,i+1)
        plt.imshow(gen_imgs[i,:,:,0], cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

In [8]:
train(epochs=1000, batch=64)  # Train for 1000 epochs

/home/pl1/.local/lib/python3.10/site-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")
E0000 00:00:1745311724.349957  172253 meta_optimizer.cc:967] remapper failed: INVALID_ARGUMENT: Mutation::Apply error: fanout 'gradient_tape/sequential_2_1/sequential_1_2/leaky_re_lu_1_1/LeakyRelu/LeakyReluGrad' exist for missing node 'sequential_2_1/sequential_1_2/conv2d_1/BiasAdd'.


0 | D loss: 0.6833 | G loss: 0.6906
